In [19]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import textwrap
from langchain_community.document_loaders import TextLoader

In [20]:
loader = DirectoryLoader('entry-data', glob="*.txt", show_progress=True)
docs = loader.load()
print(docs)

100%|██████████| 1/1 [00:00<00:00, 78.69it/s]

[Document(page_content="Date:May 28, 2024 - 9:00 PM Content:Today was another challenging day. I had a presentation in my history class, and as much as I practiced, the stuttering took over. It felt like every word was a mountain I had to climb. I saw the pity in my classmates' eyes, and even though they try to be supportive, it still stings. Mrs. Thompson, my teacher, was patient and encouraging, but I couldn't help but feel embarrassed. I wish I could speak fluently, just for once, and not have to worry about tripping over my words. I wonder if they know how much I rehearse at home, trying to get it right. Sometimes, it feels like my voice is my biggest enemy.\n\n--------------------------\n\nDate:May 29, 2024 - 10:30 PM Content:Today was slightly better. I had a therapy session with Dr. Miller. We worked on some breathing exercises and techniques to manage my speech blocks. It helps a bit, but it’s exhausting. We also talked about the anxiety that comes with speaking in public. Dr. 

In [21]:
print(docs[0])

page_content="Date:May 28, 2024 - 9:00 PM Content:Today was another challenging day. I had a presentation in my history class, and as much as I practiced, the stuttering took over. It felt like every word was a mountain I had to climb. I saw the pity in my classmates' eyes, and even though they try to be supportive, it still stings. Mrs. Thompson, my teacher, was patient and encouraging, but I couldn't help but feel embarrassed. I wish I could speak fluently, just for once, and not have to worry about tripping over my words. I wonder if they know how much I rehearse at home, trying to get it right. Sometimes, it feels like my voice is my biggest enemy.\n\n--------------------------\n\nDate:May 29, 2024 - 10:30 PM Content:Today was slightly better. I had a therapy session with Dr. Miller. We worked on some breathing exercises and techniques to manage my speech blocks. It helps a bit, but it’s exhausting. We also talked about the anxiety that comes with speaking in public. Dr. Miller sug

In [22]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(page_content="Date:May 28, 2024 - 9:00 PM Content:Today was another challenging day. I had a presentation in my history class, and as much as I practiced, the stuttering took over. It felt like every word was a mountain I had to climb. I saw the pity in my classmates' eyes, and even though they try to be supportive, it still stings. Mrs. Thompson, my teacher, was patient and encouraging, but I couldn't help but feel embarrassed. I wish I could speak fluently, just for once, and not have to worry about tripping over my words. I wonder if they know how much I rehearse at home, trying to get it right. Sometimes, it feels like my voice is my biggest enemy.\n\n--------------------------", metadata={'source': 'entry-data/data.txt'}),
 Document(page_content='Date:May 29, 2024 - 10:30 PM Content:Today was slightly better. I had a therapy session with Dr. Miller. We worked on some breathing exercises and techniques to manage my speech blocks. It helps a bit, but it’s exhausting. We al

In [23]:
documents=text_splitter.split_documents(docs)
documents

[Document(page_content="Date:May 28, 2024 - 9:00 PM Content:Today was another challenging day. I had a presentation in my history class, and as much as I practiced, the stuttering took over. It felt like every word was a mountain I had to climb. I saw the pity in my classmates' eyes, and even though they try to be supportive, it still stings. Mrs. Thompson, my teacher, was patient and encouraging, but I couldn't help but feel embarrassed. I wish I could speak fluently, just for once, and not have to worry about tripping over my words. I wonder if they know how much I rehearse at home, trying to get it right. Sometimes, it feels like my voice is my biggest enemy.\n\n--------------------------", metadata={'source': 'entry-data/data.txt'}),
 Document(page_content='Date:May 29, 2024 - 10:30 PM Content:Today was slightly better. I had a therapy session with Dr. Miller. We worked on some breathing exercises and techniques to manage my speech blocks. It helps a bit, but it’s exhausting. We al

In [24]:
db=FAISS.from_documents(documents[:30],OllamaEmbeddings())

In [25]:
db

In [27]:
query="jake"
result=db.similarity_search(query)
result[0].page_content

'Date:June 1, 2024 - 11:00 PM Content:It’s the weekend, and I spent most of the day at home. I decided to give the support group a try and joined an online meeting. It was strange at first, seeing others who share my struggle. But hearing their stories made me feel less alone. Some of them have been stuttering for years and have found ways to live with it, even thrive despite it. I spoke up towards the end, and though my voice trembled and my words caught, I felt a sense of solidarity. We all understood each other’s pain and triumphs. I’m glad I took the step to join. It gave me a bit of hope, something I’ve been needing.'

In [28]:
llm=Ollama(model="llama2")
llm

Ollama()

In [33]:
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 

Humanize your answers.
<context>
{context}
</context>
Question: {input}""")

#TODO: Write a better prompt for a sutiable context and response

In [34]:
document_chain=create_stuff_documents_chain(llm,prompt)
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x326b9e7e0>)

In [35]:
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [36]:
response=retrieval_chain.invoke({"input":""})
formatted_response = textwrap.fill(response['answer'], width=80)

print(formatted_response)

# paragraphs = response['answer'].strip().split("\n\n")

# # Print each paragraph separately
# for paragraph in paragraphs:
#     print(paragraph)
#     print()

Wow, you've been through a tough week, haven't you? *empathetic nod* It takes a
lot of courage to share your experiences with stuttering and anxiety in front of
me, and I want you to know that I'm here for you, always. *supportive smile*
Now, let's talk about your performance on May 29th. *inquiring gaze* From what
you've shared, it seems like you had a bit of a breakthrough during your therapy
session with Dr. Miller. You mentioned learning some breathing exercises and
techniques to help manage your speech blocks, which is great! It's important to
have strategies in place to cope with stuttering in different situations.
*nodding*  You also mentioned practicing speaking in front of a mirror, which is
a great way to become more comfortable with the sound of your own voice and to
work on your fluency. It's impressive that you found the courage to try
something new and challenging, even if it felt awkward at first. *smiling*
Overall, it sounds like you're taking some great steps towards i